In [1]:
import pandas as pd 
import numpy as np
import math
import keras
import tensorflow as tf
print(pd.__version__)
import progressbar
import os
from os import listdir

1.2.3


## Print Dependencies



Dependences are fundamental to record the computational environment.

In [2]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p pandas,keras,numpy,math,tensorflow,matplotlib,h5py

# date
print (" ")
%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

pandas    : 1.2.3
keras     : 2.4.3
numpy     : 1.19.5
math      : unknown
tensorflow: 2.4.1
matplotlib: not installed
h5py      : 2.10.0

Compiler    : Clang 12.0.0 (clang-1200.0.32.29)
OS          : Darwin
Release     : 19.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

 
Last updated: Tue Mar 30 2021 14:08:10CEST



## Load of the data

   You can also load all of them! Writing "all_data"

In [3]:
from process import loaddata
class_data = loaddata("../data/{}.csv".format('probability'))

In [4]:
class_data = class_data[class_data[:,0] > 0.0001]

In [5]:
np.random.shuffle(class_data)
y = class_data[:,0]
#y2 = class_data[:,-1]
x = class_data[:,1:14]

In [6]:
y[0]

0.20746096689629911

In [7]:
x[0]

array([-4.28377189e-01, -1.84104467e-01,  8.00204362e-01, -3.30019048e-01,
        5.31816209e-01,  1.06882542e-01,  3.06000000e+00,  3.93999998e+09,
       -4.09324611e-01,  1.37670373e-01, -5.39737149e-04,  4.30267198e-01,
        5.56999932e-03])

In [8]:
#print(y2.shape)
#print(y1.shape)
#print(y.shape)

In [9]:
x.shape

(135911, 13)

In [10]:
train_split = 0.75
train_limit = int(len(y)*train_split)
print("Training sample: {0} \nValuation sample: {1}".format(train_limit, len(y)-train_limit))

Training sample: 101933 
Valuation sample: 33978


In [11]:
x_train = x[:train_limit]
x_val = x[train_limit:]

y_train = y[:train_limit]
y_val = y[train_limit:]

## Model Build

In [12]:
from keras.models import Sequential
from keras.layers.core import Dense
import keras.backend as K
from keras import optimizers
from keras import models
from keras import layers
from keras.layers.normalization import BatchNormalization

In [16]:
from tensorflow.keras import backend as K

def root_mean_squared_log_error(y, y_pred):
    return K.sqrt(K.mean(K.square(K.log(1+y_pred) - K.log(1+y))))

def maape(y, y_pred):
    return K.mean(K.mean(tf.math.atan(K.abs((y-y_pred)/y))))

In [17]:
def build_model() :
    model = models.Sequential()
    model.add (BatchNormalization(input_dim = 13))
    model.add (layers.Dense (16 , activation = "sigmoid"))
    model.add (layers.Dense (32, activation = "relu"))
    model.add (layers.Dense (64 , activation = "relu"))
    model.add (layers.Dense (128 , activation = "relu"))
    model.add (layers.Dense (64, activation = "relu"))
    model.add (layers.Dense (32 , activation = "relu"))
    model.add (layers.Dense (16, activation = "relu"))
    model.add (layers.Dense (12 , activation = "relu"))
    model.add (layers.Dense (1 , activation = "sigmoid"))
    model.compile(optimizer = "adam" , loss = maape , metrics = ["mape"])
    return model

In [18]:
model = build_model ()
history = model.fit ( x_train, y_train, epochs = 10000, batch_size = 100000 , validation_data = (x_val, y_val) )
model.save("../models/classifier/{}_nodropout.h5".format('probability'))

Epoch 1/10000
2/2 [==============================] - 2s 528ms/step - loss: 1.0349 - mape: 740.8418 - val_loss: 1.0171 - val_mape: 754.2380
Epoch 2/10000
2/2 [==============================] - 0s 81ms/step - loss: 1.0191 - mape: 722.4427 - val_loss: 1.0016 - val_mape: 737.8022
Epoch 3/10000
2/2 [==============================] - 0s 85ms/step - loss: 1.0085 - mape: 710.7809 - val_loss: 0.9847 - val_mape: 720.7648
Epoch 4/10000
2/2 [==============================] - 0s 99ms/step - loss: 0.9963 - mape: 696.5256 - val_loss: 0.9624 - val_mape: 697.4182
Epoch 5/10000
2/2 [==============================] - 0s 93ms/step - loss: 0.9790 - mape: 678.7534 - val_loss: 0.9277 - val_mape: 667.3628
Epoch 6/10000
2/2 [==============================] - 0s 113ms/step - loss: 0.9552 - mape: 653.5085 - val_loss: 0.8812 - val_mape: 628.5167
Epoch 7/10000
2/2 [==============================] - 0s 111ms/step - loss: 0.9214 - mape: 622.4125 - val_loss: 0.8176 - val_mape: 580.6495
Epoch 8/10000
2/2 [============

Epoch 60/10000
2/2 [==============================] - 0s 106ms/step - loss: 0.2948 - mape: 47.0738 - val_loss: 0.3723 - val_mape: 77.8824
Epoch 61/10000
2/2 [==============================] - 0s 118ms/step - loss: 0.2953 - mape: 50.4146 - val_loss: 0.3643 - val_mape: 74.1360
Epoch 62/10000
2/2 [==============================] - 0s 80ms/step - loss: 0.2927 - mape: 48.7091 - val_loss: 0.3338 - val_mape: 62.8343
Epoch 63/10000
2/2 [==============================] - 0s 82ms/step - loss: 0.2951 - mape: 45.5099 - val_loss: 0.3590 - val_mape: 71.2749
Epoch 64/10000
2/2 [==============================] - 0s 91ms/step - loss: 0.2914 - mape: 47.4419 - val_loss: 0.3681 - val_mape: 74.8248
Epoch 65/10000
2/2 [==============================] - 0s 109ms/step - loss: 0.2926 - mape: 48.2713 - val_loss: 0.3365 - val_mape: 64.0389
Epoch 66/10000
2/2 [==============================] - 0s 105ms/step - loss: 0.2914 - mape: 45.1401 - val_loss: 0.3493 - val_mape: 68.7781
Epoch 67/10000
2/2 [=================

2/2 [==============================] - 0s 119ms/step - loss: 0.2709 - mape: 34.6301 - val_loss: 0.2886 - val_mape: 40.0411
Epoch 120/10000
2/2 [==============================] - 0s 108ms/step - loss: 0.2673 - mape: 35.0311 - val_loss: 0.3219 - val_mape: 47.7819
Epoch 121/10000
2/2 [==============================] - 0s 121ms/step - loss: 0.2734 - mape: 38.2160 - val_loss: 0.2800 - val_mape: 37.3288
Epoch 122/10000
2/2 [==============================] - 0s 117ms/step - loss: 0.2722 - mape: 34.4681 - val_loss: 0.2906 - val_mape: 40.1683
Epoch 123/10000
2/2 [==============================] - 0s 109ms/step - loss: 0.2653 - mape: 34.7697 - val_loss: 0.3293 - val_mape: 49.7613
Epoch 124/10000
2/2 [==============================] - 0s 97ms/step - loss: 0.2792 - mape: 39.6330 - val_loss: 0.2814 - val_mape: 37.5594
Epoch 125/10000
2/2 [==============================] - 0s 98ms/step - loss: 0.2677 - mape: 34.0036 - val_loss: 0.2816 - val_mape: 37.4138
Epoch 126/10000
2/2 [========================

2/2 [==============================] - 0s 111ms/step - loss: 0.2595 - mape: 31.2001 - val_loss: 0.3086 - val_mape: 41.9029
Epoch 179/10000
2/2 [==============================] - 0s 110ms/step - loss: 0.2725 - mape: 36.7980 - val_loss: 0.2896 - val_mape: 37.8035
Epoch 180/10000
2/2 [==============================] - 0s 120ms/step - loss: 0.2583 - mape: 33.6404 - val_loss: 0.2719 - val_mape: 31.8736
Epoch 181/10000
2/2 [==============================] - 0s 112ms/step - loss: 0.2693 - mape: 31.5702 - val_loss: 0.2669 - val_mape: 32.1632
Epoch 182/10000
2/2 [==============================] - 0s 129ms/step - loss: 0.2556 - mape: 30.8321 - val_loss: 0.2939 - val_mape: 38.7878
Epoch 183/10000
2/2 [==============================] - 0s 122ms/step - loss: 0.2611 - mape: 34.2092 - val_loss: 0.2716 - val_mape: 33.7377
Epoch 184/10000
2/2 [==============================] - 0s 112ms/step - loss: 0.2508 - mape: 31.0822 - val_loss: 0.2672 - val_mape: 31.6290
Epoch 185/10000
2/2 [======================

2/2 [==============================] - 0s 111ms/step - loss: 0.2360 - mape: 27.7569 - val_loss: 0.2803 - val_mape: 35.3891
Epoch 238/10000
2/2 [==============================] - 0s 114ms/step - loss: 0.2529 - mape: 32.1484 - val_loss: 0.2570 - val_mape: 30.7750
Epoch 239/10000
2/2 [==============================] - 0s 103ms/step - loss: 0.2362 - mape: 28.4185 - val_loss: 0.2553 - val_mape: 30.1450
Epoch 240/10000
2/2 [==============================] - 0s 95ms/step - loss: 0.2359 - mape: 27.9999 - val_loss: 0.2557 - val_mape: 30.0172
Epoch 241/10000
2/2 [==============================] - 0s 89ms/step - loss: 0.2364 - mape: 27.8896 - val_loss: 0.2562 - val_mape: 30.2069
Epoch 242/10000
2/2 [==============================] - 0s 69ms/step - loss: 0.2364 - mape: 28.0448 - val_loss: 0.2628 - val_mape: 31.9601
Epoch 243/10000
2/2 [==============================] - 0s 84ms/step - loss: 0.2394 - mape: 29.3767 - val_loss: 0.2534 - val_mape: 29.0352
Epoch 244/10000
2/2 [==========================

...

Epoch 1122/10000
2/2 [==============================] - 0s 134ms/step - loss: 0.1637 - mape: 18.1775 - val_loss: 0.1852 - val_mape: 21.1366
Epoch 1123/10000
2/2 [==============================] - 0s 108ms/step - loss: 0.1857 - mape: 20.6124 - val_loss: 0.1845 - val_mape: 21.0710
Epoch 1124/10000
2/2 [==============================] - 0s 102ms/step - loss: 0.1850 - mape: 20.5636 - val_loss: 0.1620 - val_mape: 18.5330
Epoch 1125/10000
2/2 [==============================] - 0s 101ms/step - loss: 0.1604 - mape: 17.7852 - val_loss: 0.1708 - val_mape: 19.2750
Epoch 1126/10000
2/2 [==============================] - 0s 106ms/step - loss: 0.1678 - mape: 18.3966 - val_loss: 0.1688 - val_mape: 18.6455
Epoch 1127/10000
2/2 [==============================] - 0s 105ms/step - loss: 0.1668 - mape: 18.0595 - val_loss: 0.1699 - val_mape: 18.4175
Epoch 1128/10000
2/2 [==============================] - 0s 103ms/step - loss: 0.1690 - mape: 17.9705 - val_loss: 0.1592 - val_mape: 17.5098
Epoch 1129/10000
2/2

2/2 [==============================] - 0s 108ms/step - loss: 0.1569 - mape: 17.4088 - val_loss: 0.1598 - val_mape: 17.3634
Epoch 1181/10000
2/2 [==============================] - 0s 105ms/step - loss: 0.1580 - mape: 16.9316 - val_loss: 0.1657 - val_mape: 18.8169
Epoch 1182/10000
2/2 [==============================] - 0s 98ms/step - loss: 0.1640 - mape: 18.3246 - val_loss: 0.1601 - val_mape: 17.9170
Epoch 1183/10000
2/2 [==============================] - 0s 91ms/step - loss: 0.1578 - mape: 17.3949 - val_loss: 0.1560 - val_mape: 16.7945
Epoch 1184/10000
2/2 [==============================] - 0s 78ms/step - loss: 0.1547 - mape: 16.4645 - val_loss: 0.1500 - val_mape: 16.3843
Epoch 1185/10000
2/2 [==============================] - 0s 72ms/step - loss: 0.1489 - mape: 16.0598 - val_loss: 0.1782 - val_mape: 20.9201
Epoch 1186/10000
2/2 [==============================] - 0s 85ms/step - loss: 0.1769 - mape: 20.5467 - val_loss: 0.1590 - val_mape: 17.1321
Epoch 1187/10000
2/2 [====================

Epoch 1239/10000
2/2 [==============================] - 0s 84ms/step - loss: 0.1742 - mape: 18.3242 - val_loss: 0.1635 - val_mape: 17.7559
Epoch 1240/10000
2/2 [==============================] - 0s 80ms/step - loss: 0.1611 - mape: 17.1862 - val_loss: 0.1855 - val_mape: 21.0068
Epoch 1241/10000
2/2 [==============================] - 0s 94ms/step - loss: 0.1838 - mape: 20.3773 - val_loss: 0.1768 - val_mape: 20.1869
Epoch 1242/10000
2/2 [==============================] - 0s 96ms/step - loss: 0.1756 - mape: 19.5597 - val_loss: 0.1585 - val_mape: 17.1786
Epoch 1243/10000
2/2 [==============================] - 0s 100ms/step - loss: 0.1571 - mape: 16.6649 - val_loss: 0.1571 - val_mape: 17.1122
Epoch 1244/10000
2/2 [==============================] - 0s 100ms/step - loss: 0.1548 - mape: 16.4329 - val_loss: 0.1647 - val_mape: 18.3551
Epoch 1245/10000
2/2 [==============================] - 0s 109ms/step - loss: 0.1653 - mape: 17.9491 - val_loss: 0.1733 - val_mape: 19.8335
Epoch 1246/10000
2/2 [==

Epoch 1298/10000
2/2 [==============================] - 0s 90ms/step - loss: 0.1434 - mape: 15.3781 - val_loss: 0.1479 - val_mape: 16.7606
Epoch 1299/10000
2/2 [==============================] - 0s 96ms/step - loss: 0.1479 - mape: 16.0968 - val_loss: 0.1673 - val_mape: 19.3714
Epoch 1300/10000
2/2 [==============================] - 0s 110ms/step - loss: 0.1662 - mape: 18.4352 - val_loss: 0.1497 - val_mape: 16.7459
Epoch 1301/10000
2/2 [==============================] - 0s 101ms/step - loss: 0.1489 - mape: 16.0234 - val_loss: 0.1588 - val_mape: 17.5519
Epoch 1302/10000
2/2 [==============================] - 0s 105ms/step - loss: 0.1564 - mape: 16.4981 - val_loss: 0.1635 - val_mape: 19.7721
Epoch 1303/10000
2/2 [==============================] - 0s 115ms/step - loss: 0.1616 - mape: 18.3327 - val_loss: 0.1531 - val_mape: 17.9839
Epoch 1304/10000
2/2 [==============================] - 0s 106ms/step - loss: 0.1509 - mape: 16.6401 - val_loss: 0.1630 - val_mape: 18.2531
Epoch 1305/10000
2/2 [

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

accuracy = history.history['mape']
val_accuracy = history.history['val_mape']


epochs = range(1, len(loss) + 1)
fig, ax1 = plt.subplots()

l1 = ax1.plot(epochs, loss, 'bo', label='Training loss')
vl1 = ax1.plot(epochs, val_loss, 'b', label='Validation loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss (mape))')

ax2 = ax1.twinx()
ac2= ax2.plot(epochs, accuracy, 'o', c="red", label='Training acc')
vac2= ax2.plot(epochs, val_accuracy, 'r', label='Validation acc')
ax2.set_ylabel('mape')

lns = l1 + vl1 + ac2 + vac2
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc="center right")
fig.tight_layout()
#fig.savefig("acc+loss_drop.pdf")
fig.show()

In [ ]:
plt.clf()   # clear figure

accuracy = history.history['mape']
val_accuracy = history.history['val_mape']

plt.plot(epochs, accuracy, 'bo', label='Training acc')
plt.plot(epochs, val_accuracy, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Probability density distribution

In [ ]:
y

In [ ]:
y_pred = model.predict(x)

In [ ]:
y_pred

In [ ]:
def probability(prob_array, bins):
    prob_array = np.array(prob_array)
    plt.hist(prob_array, bins, histtype=u'step', density=True)
    plt.show()

In [ ]:
from matplotlib import pyplot
y = np.array(y)
bins = np.linspace(0, 0.6, 100)
pyplot.hist(y, bins, color = 'indianred', alpha=0.5, label='Osiris')
pyplot.hist(y_pred, bins, color = 'mediumslateblue', alpha=0.5, label='NN')
pyplot.legend(loc='upper right')
pyplot.xlabel('Probability')
pyplot.show()